# Introduction
In my [previous work](https://www.kaggle.com/code/lazygene/exploring-psychiatric-disorders-eeg-dataset) I explored the dataset and performed multi-class classification. Here, I concentrate on binary classification disorder vs. healthy controls and report feature importance for the best performing models during 10-fold cross-validation.



# Import libraries

In [23]:
import os
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.patheffects as PathEffects
import pickle
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import classification_report
from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from copy import deepcopy
from tqdm import tqdm
import mne

# Read Data

In [24]:
df = pd.read_csv('../Datasets/EEG_data.csv')
df.head()

,no.,sex,age,eeg.date,education,IQ,main.disorder,specific.disorder,AB.A.delta.a.FP1,AB.A.delta.b.FP2,...,COH.F.gamma.o.Pz.p.P4,COH.F.gamma.o.Pz.q.T6,COH.F.gamma.o.Pz.r.O1,COH.F.gamma.o.Pz.s.O2,COH.F.gamma.p.P4.q.T6,COH.F.gamma.p.P4.r.O1,COH.F.gamma.p.P4.s.O2,COH.F.gamma.q.T6.r.O1,COH.F.gamma.q.T6.s.O2,COH.F.gamma.r.O1.s.O2
0,1,M,57.0,2012.8.30,NaN,NaN,Addictive disorder,Alcohol use disorder,35.998557,21.717375,...,55.989192,16.739679,23.452271,45.678820,30.167520,16.918761,48.850427,9.422630,34.507082,28.613029
1,2,M,37.0,2012.9.6,6.0,120.0,Addictive disorder,Alcohol use disorder,13.425118,11.002916,...,45.595619,17.510824,26.777368,28.201062,57.108861,32.375401,60.351749,13.900981,57.831848,43.463261
2,3,M,32.0,2012.9.10,16.0,113.0,Addictive disorder,Alcohol use disorder,29.941780,27.544684,...,99.475453,70.654171,39.131547,69.920996,71.063644,38.534505,69.908764,27.180532,64.803155,31.485799
3,4,M,35.0,2012.10.8,18.0,126.0,Addictive disorder,Alcohol use disorder,21.496226,21.846832,...,59.986561,63.822201,36.478254,47.117006,84.658376,24.724096,50.299349,35.319695,79.822944,41.141873
4,5,M,36.0,2012.10.18,16.0,112.0,Addictive disorder,Alcohol use disorder,37.775667,33.607679,...,61.462720,59.166097,51.465531,58.635415,80.685608,62.138436,75.888749,61.003944,87.455509,70.531662


Fix typo in specific.disorder:

Obsessive compulsi**t**ve disorder to Obsessive compulsive disorder

# Missing data

Extract separation column between PSD and FC data.

In [25]:
missing = df.isna().sum()
sep_col = missing[missing == df.shape[0]].index[0]
sep_col

'Unnamed: 122'

In [26]:
missing[missing > 0]

education        15
IQ               13
Unnamed: 122    945
dtype: int64

In [27]:
educ_na = df[df['education'].isna()]
iq_na = df[df['IQ'].isna()]
educ_iq_na = pd.concat([educ_na, iq_na]).drop_duplicates()
educ_iq_na

,no.,sex,age,eeg.date,education,IQ,main.disorder,specific.disorder,AB.A.delta.a.FP1,AB.A.delta.b.FP2,...,COH.F.gamma.o.Pz.p.P4,COH.F.gamma.o.Pz.q.T6,COH.F.gamma.o.Pz.r.O1,COH.F.gamma.o.Pz.s.O2,COH.F.gamma.p.P4.q.T6,COH.F.gamma.p.P4.r.O1,COH.F.gamma.p.P4.s.O2,COH.F.gamma.q.T6.r.O1,COH.F.gamma.q.T6.s.O2,COH.F.gamma.r.O1.s.O2
0,1,M,57.00,2012.8.30,NaN,NaN,Addictive disorder,Alcohol use disorder,35.998557,21.717375,...,55.989192,16.739679,23.452271,45.678820,30.167520,16.918761,48.850427,9.422630,34.507082,28.613029
17,18,M,30.00,2013.9.27,NaN,86.0,Addictive disorder,Alcohol use disorder,12.443237,12.503703,...,85.080600,53.533875,62.817411,68.040950,71.834549,48.779049,80.516443,22.049743,76.277261,45.629506
21,22,M,20.00,2014.10.23,NaN,116.0,Addictive disorder,Alcohol use disorder,28.287190,22.412264,...,33.065475,24.351780,18.764173,32.610337,38.451805,20.933501,48.626712,16.258394,40.471735,24.175359
155,156,M,18.00,2013.12.7,NaN,103.0,Addictive disorder,Behavioral addiction disorder,16.548803,19.042179,...,93.825121,79.091595,83.132315,88.900783,87.508591,76.257681,89.811513,62.739950,84.729816,86.322844
173,174,M,23.00,2015.11.14,NaN,113.0,Healthy control,Healthy control,19.050158,23.277470,...,68.005720,54.685472,70.694110,62.676608,67.705314,46.667642,66.673782,29.689482,74.787580,51.039538
269,270,M,25.00,2015.9.23,NaN,85.0,Obsessive compulsive disorder,Obsessive compulsitve disorder,6.186264,7.112847,...,72.743284,63.723838,61.257858,67.473157,77.664778,57.967084,73.898719,59.775191,81.941728,72.195171
270,271,M,34.00,2015.9.21,NaN,120.0,Obsessive compulsive disorder,Obsessive compulsitve disorder,12.784872,15.922964,...,72.409132,49.265064,41.222401,68.010480,72.429188,26.188907,68.653874,18.072318,63.285321,41.822238
279,280,M,35.00,2016.6.2,NaN,NaN,Obsessive compulsive disorder,Obsessive compulsitve disorder,21.524573,22.227615,...,90.934877,93.568209,94.185299,92.271775,96.445879,91.470862,96.851513,92.296182,96.209585,96.660561
280,281,M,37.00,2016.6.27,NaN,110.0,Obsessive compulsive disorder,Obsessive compulsitve disorder,11.971083,11.374465,...,91.150863,78.359575,64.861521,74.389105,89.863923,50.322649,75.469583,49.440177,84.022887,69.921675
281,282,M,22.00,2016.6.30,NaN,107.0,Obsessive compulsive disorder,Obsessive compulsitve disorder,12.516343,10.136242,...,96.419647,77.360976,95.156801,96.105427,87.746498,94.760227,97.579921,81.495805,86.770154,98.121870


In [28]:
drop_md = educ_iq_na['main.disorder'].value_counts().sort_index()
all_md = df['main.disorder'].value_counts().sort_index()[drop_md.index]
pd.concat([all_md, drop_md/all_md * 100], axis=1).set_axis(['all_data', 'na_percentage'], axis=1).sort_values('na_percentage', ascending=False)

,all_data,na_percentage
main.disorder,,
Obsessive compulsive disorder,46,13.043478
Addictive disorder,186,4.301075
Trauma and stress related disorder,128,3.906250
Healthy control,95,2.105263
Mood disorder,266,1.503759
Anxiety disorder,107,0.934579


We will loose too much data (13%) on patients with obsessive compulsive disorder, which is one of the smallest groups already. So we should consider filling missing data.  
Our options to fill them are:
- special value
- mean/median value
- imputation

We will fill missing data with **median values**.

In [29]:
display(df[['education', 'IQ']].agg(['mean', 'median']))
imputer = SimpleImputer(strategy='median')

,education,IQ
mean,13.43871,101.580472
median,13.00000,102.000000


# Pre-processing
To prepare data for classification we need:

1. Drop separation column, id colums (no. and eeg.date)
1. Encode target (main.disorder and specific.disorder) and categorical (sex) varables 
1. Drop target variables from features
1. Fill missing data in education and IQ columns
1. Apply log transformation to Age, PSD and FC features
1. Separate features and targets into separate subset for binary classification (disorder vs. healthy control)
1. Scale features

In [30]:
ptsd = df[df['specific.disorder'] == 'Posttraumatic stress disorder']
hc = df[df['specific.disorder'] == 'Healthy control'].sample(n=52, random_state=42)
data = pd.concat([ptsd, hc])
X = data

X = X.drop([sep_col, 'no.', 'eeg.date','main.disorder'], axis=1)

target_col = ['specific.disorder']
cat_col = ['sex']
sex_ord = df['sex'].unique()

# encoder for targets and sex
enc = OrdinalEncoder(categories=[sex_ord])
X[cat_col] = enc.fit_transform(X[cat_col])
X['specific.disorder'] = X['specific.disorder'].map({'Posttraumatic stress disorder': 1, 'Healthy control': 0})
target = X['specific.disorder']
X.drop(target_col, axis=1, inplace=True)

# fill missing data
mv_cols = ['education', 'IQ']
X[mv_cols] = imputer.fit_transform(X[mv_cols])

# numerical columns for log-transformation
logtrans_cols = X.drop(['sex', 'education', 'IQ'], axis=1).columns
X[logtrans_cols] = X[logtrans_cols].apply(pd.to_numeric, errors='coerce')


In [31]:
X, target

(     sex    age  education     IQ  AB.A.delta.a.FP1  AB.A.delta.b.FP2  \
 297  1.0  36.62       12.0   99.0         17.603385         17.243334   
 309  1.0  55.21       16.0  120.0         21.714048         19.579805   
 310  1.0  26.93       16.0  116.0         13.371076         14.028142   
 315  0.0  26.71       17.0  137.0         30.473244         13.954586   
 318  1.0  23.45       12.0   89.0         18.488575         19.603144   
 ..   ...    ...        ...    ...               ...               ...   
 137  1.0  29.04       16.0  114.0         22.737005         23.110192   
 116  1.0  25.65       18.0  118.0         20.081892         17.928614   
 130  1.0  29.86       20.0  113.0         46.306229         47.379694   
 184  0.0  27.00       18.0  130.0         22.152399         22.698280   
 121  1.0  23.66       17.0  130.0         37.450821         54.405099   
 
      AB.A.delta.c.F7  AB.A.delta.d.F3  AB.A.delta.e.Fz  AB.A.delta.f.F4  ...  \
 297        11.729942        

# Model selection
What algorithm will work best?  
We will use cross-validation with 10 folds to choose the best algorithm for classification for each disorder.

Algorithms to consider:
- Logistic Regression, ElasticNet
- SVM, Linear Kernel (to have access to feature importance)
- Random Forest
- XGBoost
- LightGBM
- CatBoost

In [32]:
def n_best (gs_result, n=1):
    """Returns nth best estimator parameters, mean score and std of it"""
    ind = np.where(gs_result['rank_test_score'] == n)[0][0]
    mean = gs_result["mean_test_score"][ind]
    std = gs_result["std_test_score"][ind]
    params = gs_result["params"][ind]
    return params, mean, std

def cache_mkdir(cache, directory, root_dir='.'):
    """Create directory and return path to it"""
    directory = directory.replace(' ', '_')
    path = os.path.join(root_dir, directory)
    if cache and not os.path.exists(path):
        os.makedirs(path)
    return path

def read_cache(cache, path, silent=False):
    """Read from cache file"""
    result = None
    if cache and os.path.exists(path):
        with open(path, 'rb') as file:
            if not silent:
                print(f'Extracted from cache ({path})')
            result = pickle.load(file)
    return result

def write_cache(cache, obj, path):
    """Write to cache"""
    if cache:
        with open(path, 'wb') as file:
            pickle.dump(obj, file) 

In [33]:
def grid_search(disorder, model, params, X, Y, random_seed=None, cache=True, cache_dir='grid_search', feature_list=None, silent=False):
    """
    Performs a grid search for hyperparameter tuning on a single disorder.

    Parameters:
    - disorder (str): Name of the disorder for caching purposes.
    - model (estimator): The model to tune.
    - params (dict): Hyperparameters for grid search.
    - X (DataFrame): Feature set for the given disorder.
    - Y (Series): Target labels for the given disorder.
    - random_seed (int, optional): Random seed for reproducibility.
    - cache (bool, optional): Whether to use caching to save results.
    - cache_dir (str, optional): Directory to store cache files.
    - feature_list (list, optional): List of features to be used.
    - silent (bool, optional): If True, suppress cache read notifications.

    Returns:
    - dict: A dictionary containing the best hyperparameters, mean score, standard deviation, and cv results.
    """

    # Set feature list if None
    if feature_list is None:
        feature_list = X.columns

    # Create cache directory if needed
    cache_mkdir(cache, cache_dir)

    # Get and create (if needed) a disorder cache directory
    disorder_folder = cache_mkdir(cache, disorder, cache_dir)

    # Get cache file path
    cache_file = os.path.join(disorder_folder, model.__class__.__name__)

    # Read file from cache
    result = read_cache(cache, cache_file, silent)
    if result is None:
        result = {}
        result['Model'] = model.__class__.__name__

        # Set seed
        if random_seed is not None:
            np.random.seed(random_seed)

        # Scale features
        scaler = StandardScaler()
        x = scaler.fit_transform(X[feature_list])

        # Perform grid search
        gs = GridSearchCV(model, params, cv=10, scoring='roc_auc', n_jobs=-1, verbose=1).fit(x, Y)

        # Get best estimator parameters, cross-validation mean score, and score std
        result['Params'], result['Mean_score'], result['Std_score'] = n_best(gs.cv_results_)

        # Save cv_results_
        result['cv_result'] = deepcopy(gs.cv_results_)

        # Cache results
        write_cache(cache, result, cache_file)

    return result

In [34]:
import warnings
warnings.filterwarnings('ignore')

## Specific disorder

In [35]:
X = pd.DataFrame(X) 
Y = pd.Series(target)

In [ ]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

# Helper function to create dictionaries
def lists_to_dict(keys, items):
    return {key: item for key, item in zip(keys, items)}

# Models names
model_names = ['EN', 'SVM', 'RF', 'XGB', 'LGBM', 'CatBoost']

# Parameter grids for grid search
param_grids = [
    {
        'l1_ratio': np.linspace(0, 1, 5),
        'C': [0.5, 1, 5, 10]
    },
    {
        'C': [0.5, 1, 5, 10]
    },
    {
        'n_estimators': [100, 300, 500],
        'max_depth': [1, 3, 6, None]
    },
    {
        'n_estimators': [100, 300, 500],
        'subsample': [0.3, 0.5, 1],
        'max_depth': [1, 3, 6, None]
    },
    {
        'n_estimators': [100, 300, 500],
        'subsample': [0.3, 0.5, 1],
        'max_depth': [1, 3, 6, None]
    },
    {
        'n_estimators': [100, 300, 500],
        'max_depth': [1, 3, 6, None]
    }  
]
param_grids = lists_to_dict(model_names, param_grids)


# Example model names
model_names = [
    "Logistic Regression",
    "SVC (Linear Kernel)",
    "Random Forest",
    "XGBoost",
    "LightGBM",
    "CatBoost"
]

# Instantiate models with all required parameters
models = [
    LogisticRegression(penalty='elasticnet', solver='saga', l1_ratio=0.5, random_state=77),
    SVC(kernel='linear', random_state=77, probability=True),
    RandomForestClassifier(
    n_estimators=100,        # Reduce the number of trees
    max_depth=10,            # Limit the depth of each tree
    min_samples_split=10,    # Minimum samples required to split a node
    min_samples_leaf=5,      # Minimum samples required at a leaf node
    max_features='sqrt',     # Use sqrt of features for best split
    bootstrap=True,          # Use bootstrapping for building trees
    random_state=77          # Seed for reproducibility
    ),
    XGBClassifier(
    n_estimators=100,         # Reduce the number of trees
    max_depth=4,              # Limit the depth of each tree
    learning_rate=0.1,        # Lower learning rate for smoother training
    subsample=0.8,            # Use 80% of data for training each tree
    colsample_bytree=0.8,     # Use 80% of features for training each tree
    reg_lambda=10,            # Add L2 regularization to penalize large weights
    reg_alpha=1,              # Add L1 regularization to reduce feature redundancy
    min_child_weight=5,       # Minimum sum of instance weights (like min samples)
    random_state=77,          # Seed for reproducibility
    use_label_encoder=False,  # Avoid warning
    eval_metric='logloss'   # Specify evaluation metric
    ),
    LGBMClassifier(
    n_estimators=100,        # Reduce the number of trees
    max_depth=4,             # Limit the depth of each tree
    learning_rate=0.1,       # Lower learning rate
    subsample=0.8,           # Use 80% of data for each tree
    colsample_bytree=0.8,    # Use 80% of features for each tree
    lambda_l1=1.0,           # Add L1 regularization
    lambda_l2=10.0,          # Add L2 regularization
    min_child_samples=10,    # Minimum number of data points in a leaf node
    random_state=77          # Seed for reproducibility
    ),
    CatBoostClassifier(
    iterations=100,          # Reduce the number of trees
    depth=4,                 # Limit the depth of each tree
    learning_rate=0.1,       # Lower learning rate
    subsample=0.8,           # Use 80% of data for training each tree
    l2_leaf_reg=10.0,        # Add L2 regularization
    random_seed=77,          # Seed for reproducibility
    logging_level='Silent'   # Suppress training logs)
    )
]

# Combine into a dictionary
models = dict(zip(model_names, models))


result={}
# Iterate over models
for name, model in models.items():
    param_grid = param_grids.get(name, {})
    result.update({f"{name}": grid_search(
        disorder='Posttraumatic stress disorder',
        model=model,
        params=param_grid,
        X=X,
        Y=target,
        random_seed=42
    )} )

Extracted from cache (grid_search/Posttraumatic_stress_disorder/LogisticRegression)
Extracted from cache (grid_search/Posttraumatic_stress_disorder/SVC)
Extracted from cache (grid_search/Posttraumatic_stress_disorder/RandomForestClassifier)
Extracted from cache (grid_search/Posttraumatic_stress_disorder/XGBClassifier)
Extracted from cache (grid_search/Posttraumatic_stress_disorder/LGBMClassifier)
Extracted from cache (grid_search/Posttraumatic_stress_disorder/CatBoostClassifier)


In [37]:
sd_results = pd.DataFrame(result)
sd_results

,Logistic Regression,SVC (Linear Kernel),Random Forest,XGBoost,LightGBM,CatBoost
Model,LogisticRegression,SVC,RandomForestClassifier,XGBClassifier,LGBMClassifier,CatBoostClassifier
Params,"{'C': 0.5, 'l1_ratio': 1.0}",{'C': 0.5},"{'max_depth': 3, 'n_estimators': 100}","{'max_depth': 3, 'n_estimators': 100, 'subsamp...","{'max_depth': 3, 'n_estimators': 100, 'subsamp...","{'max_depth': 6, 'n_estimators': 300}"
Mean_score,0.826,0.701333,0.782667,0.901333,0.89,0.908667
Std_score,0.119199,0.174962,0.155755,0.078304,0.08518,0.075663
cv_result,"{'mean_fit_time': [0.21592586040496825, 0.3762...","{'mean_fit_time': [0.010168337821960449, 0.011...","{'mean_fit_time': [0.29777207374572756, 0.5478...","{'mean_fit_time': [0.2883885622024536, 0.38554...","{'mean_fit_time': [0.7758616209030151, 0.80729...","{'mean_fit_time': [1.807526135444641, 3.970486..."


In [38]:

from sklearn.metrics import roc_auc_score
from sklearn.utils import shuffle


# Function for permutation test
def permutation_test(model, X, y, n_permutations=1000, metric=roc_auc_score,cache=True, cache_dir='p-test-parallel', silent=False):
    # Compute original performance
    model.fit(X, y)
    y_pred = model.predict_proba(X)[:, 1]
    original_score = metric(y, y_pred)
    
    # Compute performance for permuted labels
    permuted_scores = []
    for _ in range(n_permutations):
        y_permuted = shuffle(y, random_state=42)  # Shuffle labels
        model.fit(X, y_permuted)
        y_pred_permuted = model.predict_proba(X)[:, 1]
        permuted_score = metric(y_permuted, y_pred_permuted)
        permuted_scores.append(permuted_score)
    
    # Calculate p-value
    permuted_scores = np.array(permuted_scores)
    p_value = np.mean(permuted_scores >= original_score)
    
    return (original_score, permuted_scores, p_value)



In [39]:
%pip install joblib

Note: you may need to restart the kernel to use updated packages.


In [40]:
# Function to run permutation test for a single model
def run_permutation_test(name, model, X, Y, n_permutations):
    print(f"Running permutation test for {name}...")
    result = permutation_test(model, X, Y, n_permutations=n_permutations, metric=roc_auc_score)
    print(f"Completed {name}")
    return name, result

In [41]:
from joblib import Parallel, delayed

def display_p_test(models,cache=True, cache_dir="p-test-parallel",silent=False):
    
    cache_mkdir(cache, cache_dir) 
    cache_file = os.path.join(cache_dir, 'p-test-result')
    permutation_result = read_cache(cache, cache_file, silent)
    
    if permutation_result is None:
        results = Parallel(n_jobs=-1)(delayed(run_permutation_test)(name, model, X, Y, n_permutations=100) for name, model in models.items())
        permutation_result = dict(results)
        
    for name, result in permutation_result.items():
        original_score, permuted_scores, p_value = result
        print(f"Model: {name}")
        print(f"Original AUC: {original_score}")
        print(f"Permuted AUC: {permuted_scores}")
        print(f"P-value: {p_value}\n")
    
    write_cache(cache, permutation_result, cache_file)
    
    return permutation_result

In [42]:
display_p_test(models)

Extracted from cache (p-test-parallel/p-test-result)
Model: Logistic Regression
Original AUC: 0.9826183431952663
Permuted AUC: [0.96967456 0.96967456 0.96967456 0.96967456 0.96967456 0.96967456
 0.96967456 0.96967456 0.96967456 0.96967456 0.96967456 0.96967456
 0.96967456 0.96967456 0.96967456 0.96967456 0.96967456 0.96967456
 0.96967456 0.96967456 0.96967456 0.96967456 0.96967456 0.96967456
 0.96967456 0.96967456 0.96967456 0.96967456 0.96967456 0.96967456
 0.96967456 0.96967456 0.96967456 0.96967456 0.96967456 0.96967456
 0.96967456 0.96967456 0.96967456 0.96967456 0.96967456 0.96967456
 0.96967456 0.96967456 0.96967456 0.96967456 0.96967456 0.96967456
 0.96967456 0.96967456 0.96967456 0.96967456 0.96967456 0.96967456
 0.96967456 0.96967456 0.96967456 0.96967456 0.96967456 0.96967456
 0.96967456 0.96967456 0.96967456 0.96967456 0.96967456 0.96967456
 0.96967456 0.96967456 0.96967456 0.96967456 0.96967456 0.96967456
 0.96967456 0.96967456 0.96967456 0.96967456 0.96967456 0.96967456
 0

{'Logistic Regression': (0.9826183431952663,
  array([0.96967456, 0.96967456, 0.96967456, 0.96967456, 0.96967456,
         0.96967456, 0.96967456, 0.96967456, 0.96967456, 0.96967456,
         0.96967456, 0.96967456, 0.96967456, 0.96967456, 0.96967456,
         0.96967456, 0.96967456, 0.96967456, 0.96967456, 0.96967456,
         0.96967456, 0.96967456, 0.96967456, 0.96967456, 0.96967456,
         0.96967456, 0.96967456, 0.96967456, 0.96967456, 0.96967456,
         0.96967456, 0.96967456, 0.96967456, 0.96967456, 0.96967456,
         0.96967456, 0.96967456, 0.96967456, 0.96967456, 0.96967456,
         0.96967456, 0.96967456, 0.96967456, 0.96967456, 0.96967456,
         0.96967456, 0.96967456, 0.96967456, 0.96967456, 0.96967456,
         0.96967456, 0.96967456, 0.96967456, 0.96967456, 0.96967456,
         0.96967456, 0.96967456, 0.96967456, 0.96967456, 0.96967456,
         0.96967456, 0.96967456, 0.96967456, 0.96967456, 0.96967456,
         0.96967456, 0.96967456, 0.96967456, 0.96967456, 0